# Multiple simulations with a single progress bar

By default Nengo displays a progress bar for every simulation that takes longer than a certain time interval. When running a large number of simulations it would be more helpful to get a single progress bar that shows the progress of that outer loop. This example shows how this can be achieved.

## A simple Nengo model

To have something that we can run inside the loop, we define a simple Nengo model. Apart from that, it is not doing anything useful.

In [1]:
import nengo

with nengo.Network() as model:
    ens = nengo.Ensemble(500, 1)

## ProgressTracker

To display a progress bar, we use the ``ProgressTracker`` class which can be imported from `nengo.utils.progress`. We also need to import ``Progress`` which holds the data of the current progress.

In [2]:
from nengo.utils.progress import ProgressTracker, Progress

To use the ``ProgressTracker``, it has to be instantiated with two arguments:

* The first argument gives the progress bar instance to display. It can be ``True`` to use the default.
* The second argument is a ``Progress`` instance that is used to track the actual progress.

The ``Progress`` instance needs to be given a task description that is displayed by the progress bar (and optionally a different description to use after the processing has finished) and a ``max_steps`` argument with the maximum number of steps in the processing. ``max_steps`` may be set to ``None`` to display a processing indicator with elapsed time without an actual progress (which is unknown in this case).

The ``ProgressTracker`` should be used as a context manager, so that the progress bar gets updated appropriately if an exception occurs during the processing. For each completed processing step the ``ProgressTracker.total_progress.step()`` method should be called.

Furthermore, we set the `progress_bar` argument of the `Simulator` to `False` to prevent an extra progress bar from popping up for each trial.

In [3]:
n_trials = 100
with ProgressTracker(True, Progress('Running trials', max_steps=n_trials)) as pt:
    for i in range(n_trials):
        with nengo.Simulator(model, progress_bar=False) as sim:
            sim.run(1.)
            pt.total_progress.step()